In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:15 http://pp

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-22 04:34:26--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.51MB/s    in 0.2s    

2021-11-22 04:34:26 (5.51 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in Shoes review data from s3.amazonaws.com into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

shoes_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), inferSchema=True, sep="\t", timestampFormat="mm/dd/yy")
shoes_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [5]:
#count rows
print(shoes_df.count())


4366916


In [6]:
#drop incomplete rows
shoes_df = shoes_df.dropna()
print(shoes_df.count())

4366324


In [7]:
#drop duplicates
shoes_df = shoes_df.dropDuplicates()
#show total counts of Shoes reviews data after dropping duplicates and incomplete rows
print(shoes_df.count())


4366324


In [8]:
shoes_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



Transform DataFrame to Table Schema

In [9]:
# Converting review_date column from string to date 
from pyspark.sql.functions import to_date
 
shoes_df1 = shoes_df.withColumn('review_date',to_date(shoes_df.review_date, 'yyyy-MM-dd'))
shoes_df1.printSchema()
shoes_df1.select("review_date").dtypes

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



[('review_date', 'date')]

In [10]:
review_id_table = shoes_df1.select(["review_id","customer_id","product_id","product_parent", "review_date"])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000E0AHJ5E1V|   12731835|B00E9PFHPK|     257352271| 2013-10-04|
|R100A5CHCA9GMG|   12123247|B00DZI79R6|     575104294| 2015-04-10|
|R100A968SZR4GH|    2481979|B00AMU1Z4I|     706513982| 2014-08-04|
|R100HVIG9VEABL|   10761726|B003GIS6FE|     634629189| 2011-05-14|
|R100ZMBT0DGFFH|   32268353|B0061RUKCE|     195365109| 2014-08-20|
|R101BSLZ8K12T6|    1796899|B0023NTWN8|     300200501| 2015-01-23|
|R101I5DOJAUSZ7|   23422005|B00QAKVP4E|      77952021| 2015-04-01|
|R101VVQ9LXBADV|   35169932|B00B9TXEUY|     898577149| 2014-05-30|
|R101VY19JHVABI|   37230772|B005CFVSZY|     268889520| 2013-10-06|
|R1026R3QQEM02Q|   28698079|B000TH4T42|     361609926| 2015-08-20|
|R102IW3CHZL3F4|   41657996|B001G8FD2K|     805441169| 2012-12-05|
|R102JNG6P6AHY7|   49541857|B0090S9KJK|     807355156| 2013-02

In [11]:
products = shoes_df1.select(["product_id","product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00E9PFHPK|Qupid Ponder-06 N...|
|B00DZI79R6|REFRESH CHERRY-02...|
|B00AMU1Z4I|Nike Women's Luna...|
|B003GIS6FE|Keen Concordia Co...|
|B0061RUKCE|LensCovers Sungla...|
|B0023NTWN8|Kamik Rocket Cold...|
|B00QAKVP4E|Gabor Lanzarote 7...|
|B00B9TXEUY|Merrell Women's J...|
|B005CFVSZY|Crocs Women's Adr...|
|B000TH4T42|Bogs Men's Classi...|
|B001G8FD2K|KEEN Kootenay Win...|
|B0090S9KJK|Qupid PROUD-09 Cu...|
|B004MOFUBY|Stride Rite Flora...|
|B0026ZQJF2|Bearpaw Women's K...|
|B00CYBLN3G|Skechers Sport Wo...|
|B003PGQ8VG|Polo Ralph Lauren...|
|B00K3HLFZE|PUMA Men's San Pa...|
|B0088QMEAM|ASICS Men's GEL-1...|
|B006ZUU446|Clarks Women's Un...|
|B003C1POY6|Propet Women's Am...|
+----------+--------------------+
only showing top 20 rows



In [12]:
customers1 = shoes_df1.select(["customer_id"]).groupby("customer_id").agg({"customer_id":"count"})
customers1.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   32605012|                 2|
|    7458575|                 1|
|   33739415|                22|
|   52884959|                 3|
|   18141648|                 1|
|    6057103|                 4|
|   10805992|                 1|
|   50621360|                 2|
|   35465009|                 1|
|    2784934|                34|
|   48396961|                 1|
|    2456076|                 1|
|   22163038|                 1|
|   11831977|                 1|
|    1513544|                 1|
|   17861769|                 3|
|   42909294|                 6|
|   23947515|                27|
|    1446779|                 1|
|   11224924|                 1|
+-----------+------------------+
only showing top 20 rows



In [13]:
customers = customers1.withColumnRenamed("count(customer_id)", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   32605012|             2|
|    7458575|             1|
|   33739415|            22|
|   52884959|             3|
|   18141648|             1|
|    6057103|             4|
|   10805992|             1|
|   50621360|             2|
|   35465009|             1|
|    2784934|            34|
|   48396961|             1|
|    2456076|             1|
|   22163038|             1|
|   11831977|             1|
|    1513544|             1|
|   17861769|             3|
|   42909294|             6|
|   23947515|            27|
|    1446779|             1|
|   11224924|             1|
+-----------+--------------+
only showing top 20 rows



In [14]:
vine_table = shoes_df1.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1000E0AHJ5E1V|          4|            0|          0|   N|
|R100A5CHCA9GMG|          1|            2|          2|   N|
|R100A968SZR4GH|          5|            0|          0|   N|
|R100HVIG9VEABL|          5|            1|          1|   N|
|R100ZMBT0DGFFH|          5|            8|         10|   N|
|R101BSLZ8K12T6|          5|            0|          1|   N|
|R101I5DOJAUSZ7|          4|            0|          0|   N|
|R101VVQ9LXBADV|          5|            1|          1|   N|
|R101VY19JHVABI|          4|            0|          0|   N|
|R1026R3QQEM02Q|          5|            0|          0|   N|
|R102IW3CHZL3F4|          5|            0|          0|   N|
|R102JNG6P6AHY7|          5|            0|          0|   N|
|R102X52FTDKNGJ|          4|            0|          0|   N|
|R10330PL0P9BSY|          4|            

Write DataFrame to RDS

In [15]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<rds endpoint>:5432/databasename"
config = {"user":"root",
          "password": "password",
          "driver":"org.postgresql.Driver"}

In [16]:
# Write DataFrame to table

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table_shoes', mode=mode, properties=config)

In [17]:
# Write DataFrame to table

products.write.jdbc(url=jdbc_url, table='products_shoes', mode=mode, properties=config)

In [18]:
# Write DataFrame to table

customers.write.jdbc(url=jdbc_url, table='customers_shoes', mode=mode, properties=config)

In [19]:
# Write DataFrame to table

vine_table.write.jdbc(url=jdbc_url, table='vine_table_shoes', mode=mode, properties=config)